The purpose of this script is to upload a screen to OMERO on the basis of the filenames, for example when number of images is not equal per well, then this script will extract the well from the filename

In [ ]:
from omero.gateway import BlitzGateway
import ezomero
import os
import re
from dotenv import load_dotenv
import omero
from omero.model import PlateI, ScreenI, WellI, WellSampleI, ImageI
from omero.rtypes import rint, rlong, rstring

In [ ]:
load_dotenv(override=True)

conn = ezomero.connect(user=os.environ.get("USER_NAME"),password=os.environ.get("PASSWORD"),group="system",host=os.environ.get("HOST"),port=os.environ.get("PORT"),secure=True)
connection_status = conn.connect()
if connection_status:
    print("Connected to OMERO Server")
else:
    print("Connection to OMERO Server Failed")
conn.c.enableKeepAlive(60)

In [ ]:
#id = ezomero.ezimport(conn, 'D:\\Surfdrive\\Projects\\Mazene Hochane\Exp3_FUCCI\\25475_PDLO_Day7_Fixed\\export-ome\\17-03-25_PDLO_FUCCI_Fixed_001_A1_0001.ome.tif')
id = ezomero.ezimport(conn, 'D:\\Surfdrive\\Projects\\Mazene Hochane\Exp3_FUCCI\\25475_PDLO_Day7_Fixed\\17-03-25_PDLO_FUCCI_Fixed_001_text.tif')
print("Image ID: ", id)

In [ ]:
def extract_well_info(filename):
    """
    Extract well position (e.g., A1) from filename.
    Modify this regex pattern to match your specific filename format.
    """
    # Example pattern for "17-03-25_PDLO_FUCCI_Fixed_001_A1_0001.ome.tif"
    pattern = r'.*_([A-Z]\d+)_\d+\.ome\.tif'
    match = re.search(pattern, filename)
    if match:
        well_pos = match.group(1)
        # Convert well position to row and column indices
        row = ord(well_pos[0]) - ord('A')  # A=0, B=1, etc.
        col = int(well_pos[1:]) - 1  # 1-based to 0-based
        return well_pos, row, col
    return None, None, None

def create_screen(conn, screen_name):
    """Create a new screen with the given name."""
    screen = ScreenI()
    screen.name = rstring(screen_name)
    return conn.getUpdateService().saveAndReturnObject(screen)


# Function to create a plate in a screen
def create_plate(conn, plate_name, screen_id=None):
    """Create a new plate with the given name, optionally in a screen."""
    plate = PlateI()
    plate.name = rstring(plate_name)
    # Set naming conventions
    plate.columnNamingConvention = rstring("number")
    plate.rowNamingConvention = rstring("letter")
    plate = conn.getUpdateService().saveAndReturnObject(plate)
    
    # Link plate to screen if screen_id is provided
    if screen_id:
        link = omero.model.ScreenPlateLinkI()
        link.parent = ScreenI(screen_id, False)
        link.child = PlateI(plate.id.val, False)
        conn.getUpdateService().saveObject(link)
    
    return plate

# Function to add an image to a well
def add_image_to_well(conn, image_id, plate_id, row, col):
    """Add an image to a specific well in a plate."""
    # Create new well
    well = WellI()
    well.plate = PlateI(plate_id, False)
    well.row = rint(row)
    well.column = rint(col)
    
    # Initialize the wellSample collection
    well.initWellSamples()
    
    # Create well sample
    ws = WellSampleI()
    ws.image = ImageI(image_id, False)
    ws.well = well
    well.addWellSample(ws)
    
    return conn.getUpdateService().saveAndReturnObject(well)

# Function to add images to a plate
def add_images_to_plate(conn, images_by_well, plate_id):
    """
    Add multiple images to wells in a plate.
    
    Args:
        conn: OMERO connection
        images_by_well: Dictionary with (row, col) as keys and lists of image IDs as values
        plate_id: ID of the plate to add wells to
        
    Returns:
        Number of wells created
    """
    update_service = conn.getUpdateService()
    wells_created = 0
    
    for (row, col), image_ids in images_by_well.items():
        # Create new well
        well = WellI()
        well.plate = PlateI(plate_id, False)
        well.row = rint(row)
        well.column = rint(col)
        
        try:
            # Add all images to this well
            for image_id in image_ids:
                ws = WellSampleI()
                ws.image = ImageI(image_id, False)
                ws.well = well
                well.addWellSample(ws)
                
            # Save the well
            update_service.saveObject(well)
            wells_created += 1
            print(f"Added {len(image_ids)} images to well at row {row}, column {col}")
            
        except Exception as e:
            print(f"Error adding images to well at row {row}, column {col}: {e}")
    
    return wells_created

In [ ]:
def create_screen_from_images(conn, image_dir, screen_name, plate_name):
    """
    Process images in a directory and create a screen/plate structure.
    Handles multiple images per well.
    """
    # Create screen
    screen = create_screen(conn, screen_name)
    screen_id = screen.id.val
    print(f"Created screen: {screen_name} (ID: {screen_id})")
    
    # Create plate
    plate = create_plate(conn, plate_name, screen_id)
    plate_id = plate.id.val
    print(f"Created plate: {plate_name} (ID: {plate_id})")
    
    # Dictionary to collect images per well
    images_by_well = {}
    
    # Process all files in the directory
    for filename in os.listdir(image_dir):
        if filename.endswith('.ome.tif'):
            filepath = os.path.join(image_dir, filename)
            
            # Extract well information
            well_pos, row, col = extract_well_info(filename)
            if well_pos is None:
                print(f"Could not extract well info from: {filename}")
                continue
            
            # Import the image
            print(f"Importing: {filename} to well {well_pos}")
            image_id = ezomero.ezimport(conn, filepath)[0]
            
            # Add to images_by_well dictionary
            if (row, col) not in images_by_well:
                images_by_well[(row, col)] = []
            images_by_well[(row, col)].append(image_id)
    
    # Add all images to wells
    wells_created = add_images_to_plate(conn, images_by_well, plate_id)
    print(f"Created {wells_created} wells with images")
    
    return screen_id, plate_id

In [ ]:
image_dir = r'D:\Surfdrive\Projects\Mazene Hochane\Exp3_FUCCI\25475_PDLO_Day7_Fixed\export-ome'
screen_name = "PDLO_FUCCI_Fixed"
plate_name = "25475_PDLO_Day7_Fixed"

screen_id, plate_id = create_screen_from_images(conn, image_dir, screen_name, plate_name)
print(f"Complete! Created screen ID: {screen_id}, plate ID: {plate_id}")